In [1]:
# signature = "{'dag_id': 'smc_daily_import', 'task_id': 'weblog_index_update', 'owner': 'icarus_so', 'nb_path': '/app/ws-ic/20200622_ssl_schema/update_domain_index.ipynb', 'execution_date': '2022-03-12'}"

In [1]:
%config Completer.use_jedi = False

#### 0. Initialization

In [2]:
import argparse
from smtutil import hiveutil
import pandas as pd
import os
import shutil
import logging
import sys
import yaml

from datetime import datetime
from pyspark.sql.window import Window
import pyspark.sql.functions as F

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

# logger = logging.getLogger(__name__)
# log_format = "[%(asctime)s][%(funcName)25s:%(lineno)4d] %(message)s"
# logging.basicConfig(stream=sys.stdout, level=logging.INFO, format=log_format)

In [3]:
# from pathlib import Path
# for child in Path('/app/sna_bdmrdev01/ws-ic/hive/weblog_v2.db/').glob('*'):
#     print(datetime.fromtimestamp(child.stat().st_mtime))

In [4]:
sc, spark = hiveutil.get_sc_spark()

In [5]:
hive_db_name = 'weblog'

In [6]:
base_dir = '/app/sna_bdmrdev01/ws-ic/hive/'

ssl_data_dir = base_dir + '/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
t_domain_base_dir = '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/'

t_domain_index_dir = t_domain_base_dir + 't_domain_index' # '/t_domain_index/'
t_domain_index_backup_dir = t_domain_base_dir + '/t_domain_index_backup/'
t_domain_index_new_dir = t_domain_base_dir + '/t_domain_index_new/'

ssl_data_dir, t_domain_base_dir, t_domain_index_dir, t_domain_index_backup_dir, t_domain_index_new_dir

('/app/sna_bdmrdev01/ws-ic/hive//prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/',
 '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/',
 '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/t_domain_index_dev',
 '/app/sna_bdmrdev01/ws-ic/hive/weblog.db//t_domain_index_backup/',
 '/app/sna_bdmrdev01/ws-ic/hive/weblog.db//t_domain_index_new/')

In [7]:
# read ssl data
sdf = spark.read.parquet(ssl_data_dir)
sdf_tdi = spark.read.parquet(t_domain_index_dir)

In [8]:
# sdf_tdi.createOrReplaceTempView('sdf_tdi')
# spark.sql('REFRESH table sdf_tdi')

DataFrame[]

#### 1. Update Domain Index

In [8]:
# max_date_id = sorted([int(x.split('=')[1]) if '=' in x else -1 for x in os.listdir(ssl_data_dir)])[-1]
# max_date_id

# get max date id
max_date_id = sdf.select(F.max('date_id').alias('max_date_id')).first()['max_date_id']
max_date_id

20230205

In [17]:
# l = [i for i in range(20230117, 20230132)]
# l = l + [i for i in range(20230201, 20230207)]
# l

[20230117,
 20230118,
 20230119,
 20230120,
 20230121,
 20230122,
 20230123,
 20230124,
 20230125,
 20230126,
 20230127,
 20230128,
 20230129,
 20230130,
 20230131,
 20230201,
 20230202,
 20230203,
 20230204,
 20230205,
 20230206]

In [15]:
print(f'count: {sdf_tdi.count()}')

count: 3874099


In [19]:
# for max_date_id in l:

print(max_date_id, datetime.now())

# t domain count
sdf_t_domain_count = sdf.select('ssl_cert_domain').filter(F.col('date_id')==max_date_id).groupby('ssl_cert_domain').agg(F.count('*').alias('c'))

# add filter >= 50
sdf_t_domain_count = sdf_t_domain_count.filter(F.col('c')>=50)

# operations
sdf_t_domain_index_new = sdf_t_domain_count \
    .join(F.broadcast(sdf_tdi), sdf_t_domain_count.ssl_cert_domain == sdf_tdi.domain, 'outer')\
    .drop('o_domain_index')\
    .select(
        F.when(~F.col('ssl_cert_domain').isNull(), F.col('ssl_cert_domain')).otherwise(F.col('domain')).alias('domain'),
        F.col('domain_index').alias('o_domain_index').cast('string'),
        F.row_number().over(Window.orderBy(F.col('domain_index').asc_nulls_last())).alias('domain_index'),
)

sdf_t_domain_index_new.limit(10).show()

print(f'count: {sdf_t_domain_index_new.count()}')

# write to t domain index new
sdf_t_domain_index_new.coalesce(15).write.mode('overwrite').parquet(t_domain_index_new_dir)

# remove backup index table
try:
    shutil.rmtree(t_domain_index_backup_dir)
except FileNotFoundError:
    pass

# change current table to backup
shutil.move(t_domain_index_dir, t_domain_index_backup_dir)
# apply new table
shutil.move(t_domain_index_new_dir, t_domain_index_dir)

print(max_date_id, datetime.now())

# spark.sql('REFRESH table sdf_tdi')

20230206 2023-02-07 11:38:08.024077
+--------------------+--------------+------------+
|              domain|o_domain_index|domain_index|
+--------------------+--------------+------------+
|0:0:0:0:0:FFFF:B7...|             1|           1|
|10755729.FLS.DOUB...|             2|           2|
|             163.COM|             3|           3|
|200VOD-ADAPTIVE.A...|             4|           4|
|6522532.FLS.DOUBL...|             5|           5|
|6LOZTSC949.EXECUT...|             6|           6|
|719A4BB12674444F8...|             7|           7|
|9279533.FLS.DOUBL...|             8|           8|
|                ?125|             9|           9|
|      A.C.APPIER.NET|            10|          10|
+--------------------+--------------+------------+

count: 3877929
20230206 2023-02-07 11:39:25.470047


In [21]:
spark.stop()
sc.stop()

In [ ]:
# def main(ssl_data_dir, t_domain_base_dir):

#     # get spark
#     sc, spark = hiveutil.get_sc_spark()

#     ## path settings
#     t_domain_index_dir = t_domain_base_dir + '/t_domain_index/'
#     t_domain_index_backup_dir = t_domain_base_dir + '/t_domain_index_backup/'
#     t_domain_index_new_dir = t_domain_base_dir + '/t_domain_index_new/'
    
#     # read ssl data
#     sdf = spark.read.parquet(ssl_data_dir)
#     sdf_tdi = spark.read.parquet(t_domain_index_dir)
    
#     logging.info("Read ssl data and t domain index data")
    
#     ## ---------------- 1. Update Domain Index ----------------  ##
#     # get max date id
#     max_date_id = sdf.select(F.max('date_id').alias('max_date_id')).first()['max_date_id']
#     max_date_id

#     # t domain count
#     sdf_t_domain_count = sdf.select('ssl_cert_domain').filter(F.col('date_id')==max_date_id).groupby('ssl_cert_domain').agg(F.count('*').alias('c'))

#     ## t doamin index new
#     # t doamin index new
#     sdf_t_domain_index_new = sdf_t_domain_count \
#         .join(F.broadcast(sdf_tdi), sdf_t_domain_count.ssl_cert_domain == sdf_tdi.domain, 'outer') \
#         .drop('o_domain_index') \
#         .select(
#             F.when(~F.col('ssl_cert_domain').isNull(), F.col('ssl_cert_domain')).otherwise(F.col('domain')).alias('domain'),
#             F.col('domain_index').alias('o_domain_index'),
#             F.row_number().over(Window.orderBy(F.col('domain_index').asc_nulls_last())).alias('domain_index'),
#         )

#     logging.info("Joined new t domain index table")

#     # write new t_domain_index table
#     sdf_t_domain_index_new.coalesce(15).write.mode('overwrite').parquet(t_domain_index_new_dir)

#     logging.info("Wrote new t domain index table")

#     ## ---------------- 2. Rename New Tables ----------------  ##
#     # remove backup index table
#     try:
#         shutil.rmtree(t_domain_index_backup_dir)
#         logging.info("Removed backup t domain index table")
#     except FileNotFoundError:
#         pass

#     # change current table to backup
#     shutil.move(t_domain_index_dir, t_domain_index_backup_dir)
#     logging.info("Finish backup t domain index table")
#     # apply new table
#     shutil.move(t_domain_index_new_dir, t_domain_index_dir)
    
#     # stop spark session and spark content
#     spark.stop()
#     sc.stop()
    
#     logging.info("End")

In [ ]:
# if __name__ == '__main__':

#     # parser
#     parser = argparse.ArgumentParser(description='config')
#     parser.add_argument('--cfg', type=str, default='/app/sna_bdmrdev03/ws-simon/git_repo/weblog/ssl_to_parquet/pipeline/config_pipeline.yaml',
#                         help='.yaml config file path')
#     args = parser.parse_args()

#     # load yaml
#     fhandle = open(args.cfg,'r')
#     try:
#         cfg = yaml.safe_load(fhandle)
#     except:
#         raise Exception('no such config file')

# #     print('[Info] Configs:\n')
# #     for k, v in cfg.items():
# #         print(f'{k} : {v}')
# #     print('\n')

#     main(ssl_data_dir=cfg['ssl_data_dir'], t_domain_index_base_dir=cfg['t_domain_index_base_dir'])